In [1]:
from labs import pitch
import pynq.lib.dma
from pynq import Xlnk

overlay = pitch('/home/xilinx/E-elements_lab/labs/bitstreams/Audio.bit')

In [2]:
pAudio = overlay.audio
ina = overlay.axi_dma_a.sendchannel
inb = overlay.axi_dma_b.sendchannel
outa = overlay.axi_dma_a.recvchannel
outb = overlay.axi_dma_b.recvchannel
xlnk = Xlnk()

In [ ]:
pAudio.select_line_in()

In [ ]:
pAudio.bypass(seconds=5)

In [ ]:
pAudio.record(20)
pAudio.save("recording_2.wav")

In [ ]:
from IPython.display import Audio as IPAudio
IPAudio("/home/xilinx/jupyter_notebooks/base/audio/recording_0.wav")

In [3]:
def stretch(snd_array, factor, window_size, h):
    """ Stretches/shortens a sound, by some factor. """
    in_bufferA = xlnk.cma_array(shape=(window_size,), dtype=np.int32)
    in_bufferB = xlnk.cma_array(shape=(window_size,), dtype=np.int32)
    out_bufferA = xlnk.cma_array(shape=(window_size,), dtype=np.int32)
    out_bufferB = xlnk.cma_array(shape=(window_size,), dtype=np.int32)

    phase = np.zeros(window_size)
    hanning_window = np.hanning(window_size)
    result = np.zeros(int(len(snd_array) / factor + window_size))
    for i in np.arange(0, len(snd_array) - (window_size + h), h*factor):
        i = int(i)
        in_bufferA.physical_address = snd_array.physical_address + 0X04 * i
        in_bufferB.physical_address = snd_array.physical_address + 0X04 * (i + h)
        
        overlay.data_len(0x2000)
        overlay.start()
        
        ina.transfer(in_bufferA)
        inb.transfer(in_bufferB)
        outa.transfer(out_bufferA)
        outb.transfer(out_bufferB)

        s1 = np.fft.fft(out_bufferA)
        s2 = np.fft.fft(out_bufferB)
        phase = (phase + np.angle(s2/s1)) % 2*np.pi

        a2_rephased = np.fft.ifft(np.abs(s2)*np.exp(1j*phase))
        i2 = int(i/factor)
        result[i2: i2 + window_size] += hanning_window*a2_rephased.real
    xlnk.xlnk_reset()
    return result.astype('int16')

In [4]:
def speedx(sound_array, factor):
    """ Multiplies the sound's speed by some `factor` """
    indices = np.round( np.arange(0, len(sound_array), factor) )
    indices = indices[indices < len(sound_array)].astype(int)
    return sound_array[ indices.astype(int) ]

def pitchshift(snd_array, n, window_size=2**13, h=2**11):
    """ Changes the pitch of a sound by ``n`` semitones. """
    factor = 2**(1.0 * n / 12.0)
    stretched = stretch(snd_array, 1.0/factor, window_size, h)
    return speedx(stretched[window_size:], factor)

In [5]:
import time
import wave
import numpy as np
from IPython.display import Audio as IPAudio
np.seterr(divide='ignore', invalid='ignore')

wav_path = "/home/xilinx/jupyter_notebooks/base/audio/recording_0.wav"
with wave.open(wav_path, 'r') as wav_file:
    raw_frames = wav_file.readframes(-1)
    num_frames = wav_file.getnframes()
    num_channels = wav_file.getnchannels()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()
    
temp_buffer = np.empty((num_frames, num_channels, 4), dtype=np.uint8)
raw_bytes = np.frombuffer(raw_frames, dtype=np.uint8)
temp_buffer[:, :, :sample_width] = raw_bytes.reshape(-1, num_channels, 
                                                    sample_width)
temp_buffer[:, :, sample_width:] = \
    (temp_buffer[:, :, sample_width-1:sample_width] >> 7) * 255
frames = temp_buffer.view('<i4').reshape(temp_buffer.shape[:-1])

total_len = len(frames[:,0])
in_total = xlnk.cma_array(shape=(total_len,), dtype=np.int32)

for index in range(total_len):
    in_total[index] = frames[index,0] / 256

In [6]:
start = time.time()
wave_pitch = pitchshift(in_total, 10)
end = time.time()

print("second:  " + str(end - start))

IPAudio(data=wave_pitch, rate=sample_rate)

second:  9.89312195777893
